In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import re

# SQLite 데이터베이스 연결
conn = sqlite3.connect('nene_chicken.db')
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS stores (
    name TEXT,
    address TEXT,
    phone TEXT
)
''')

# 데이터 수집 및 저장 함수
def scrape_and_save(page):
    url = f"https://nenechicken.com/17_new/sub_shop01.asp?page={page}&ex_select=1&ex_select2=&IndexSword=&GUBUN=A"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    store_divs = soup.find_all('div', class_='shopLine')
    
    for store in store_divs:
        name = store.find('div', class_='shopName').text.strip()
        partial_address = store.find('div', class_='shopAdd').text.strip()
        phone = store.find('span', class_='tooltiptext').text.strip()
        
        # 전체 주소 추출
        map_link = store.find('div', class_='shopMap').find('a')['href']
        full_address = re.search(r"codeAddress\('(.+?)'\);", map_link).group(1)
        
        # 데이터베이스에 저장
        cursor.execute('INSERT INTO stores (name, address, phone) VALUES (?, ?, ?)',
                       (name, full_address, phone))
    
    conn.commit()

# 모든 페이지 스크래핑 (예: 1부터 82페이지까지)
for page in range(1, 83):
    scrape_and_save(page)

# 데이터베이스 연결 종료
conn.close()

# pandas DataFrame으로 데이터 확인
conn = sqlite3.connect('nene_chicken.db')
df = pd.read_sql_query("SELECT * FROM stores", conn)
print(df.head())
conn.close()

In [ ]:
df